# F-16 Absorption Model
## Ignacio Lara

### Steps
1. Initialization
  1. Populate the squadron
    * Size (API-1 and API-6)
    * Distribution of INX/EXP
    * Distribution of sortie counts/qualifications/placement in upgrades
1. Inflow new pilots
  * Distribution of B-Course vs. FAIP/PQP (including qualifications of nth-tour)
  * Distribution of API-1 vs API-6
1. Determine resources available
   * Flying Hours remaining
   * Sorties (capacity, driven by UTE and PAA)
   * IPs and/or appropriate upgrade support pilots
1. Determine sortie needs
   * Upgrade vs. CT
   * ASD for FHP accrual
1. Schedule resources for sorties
1. Attempt to fly sorties
   * Refly rate to capture ground abort, incomplete sorties, wx cancellations, etc.
   * Decrement FHP for sorties flown
1. Credit sortie completion for syllabus and/or RTM "beans"
1. Award upgrades / experiencing
1. Outflow pilots

<hr />

### 0. Setup

In [319]:
import numpy as np

rng = np.random.default_rng()

ug_names = ['MQT', 'FLUG', 'IPUG']
ug_rides = [10,     15,     8]
ug_quals = ['WM',   'FL',   'IP']

class Pilot:
    """An Air Force pilot that fills squadron billets with qualifications"""
    # Syllabus lengths (number of UP sorties)
    syllabi_rides = {ug: dur for ug, dur in zip(ug_names, ug_rides)}

    # Quals awarded when completing upgrades
    ug_awards = {ug: q for ug, q in zip(ug_names, ug_quals)}

    # Everyone gets these many sorties from FTU
    FTU_sorties = 15

    # Definition of experience
    exp_sorties = 250
    exp_qual = 'FL'

    def __init__(self, f16_sorties, tos, api_category, quals=[], ug=None):
        self.f16_sorties = self.FTU_sorties + f16_sorties
        self.tos = tos
        self.quals = quals
        self.check_experience()
        self.api_category = 1 if not self.is_exp else api_category
        self.ug = None
        if ug is not None:
            self.enroll_upgrade(ug)

    def enroll_upgrade(self, ug):
        assert self.ug is None, 'Pilot already enrolled in upgrade'
        assert ug in ug_names, 'Invalid upgrade specified'
        self.ug = ug
        self.ride_num = 0

    def fly_ug_sortie(self):
        assert self.ug is not None, 'Pilot not enrolled in any upgrade'
        self.ride_num += 1
        self.f16_sorties += 1
        if self.ride_num == self.syllabi_rides[self.ug]:
            self.award_qual(self.ug_awards[self.ug])

    def fly_spt_sortie(self):
        self.f16_sorties += 1

    def award_qual(self, qual):
        assert qual in ug_quals, 'Invalid qualification specified'
        self.quals.append(qual)
        self.ug = None

    def check_experience(self):
        self.is_exp = (self.f16_sorties >= self.exp_sorties and 
                      self.exp_qual in self.quals)

    def summarize(self):
        text = '{}-{} -- S: {} -- T: {}mo. -- Q: {}'.format('EXP' if self.is_exp else '(I)',
                                                            self.api_category,
                                                            self.f16_sorties,
                                                            self.tos,
                                                            self.quals)
        if self.ug is not None:
            text += ' -- U: {} #{}'.format(self.ug, self.ride_num)
        return text


# Helper functions
def add_INX_pilot(f16_sorties=0, tos=0, quals=[], ug=None):
    return Pilot(f16_sorties, tos, 1, quals, ug)

def add_EXP_pilot(f16_sorties=250, tos=20, api_category=1, quals=ug_quals[:-1], ug=None):
    return Pilot(f16_sorties, tos, api_category, quals, ug)

# Squadron initialization
def populate_sq(size=10, prop_INX=0.55, prop_IP=0.3, num_API6=2):
    pilots = []

    num_INX = int(size*prop_INX)
    num_EXP = size - num_INX
    num_IP = int(num_EXP*prop_IP)
    num_API1 = size - num_API6
    # Parameters
    max_TOS_INX = 24
    min_TOS_EXP = 24
    max_TOS_EXP = 32

    min_f16_sorties_EXP = 250
    max_f16_sorties_EXP = 500

    max_f16_sorties_INX = 250

    staff = num_API6
    # Add EXP pilots
    for _ in range(num_EXP):
        tos = rng.integers(min_TOS_EXP, max_TOS_EXP + 1)
        # Award sorties proportional to TOS
        sorties = int((tos - min_TOS_EXP)/(max_TOS_EXP - min_TOS_EXP) *
                  (max_f16_sorties_EXP - min_f16_sorties_EXP)) + min_f16_sorties_EXP

        quals = ug_quals[:-1]

        if staff > 0:
            api = 6
            staff -= 1
        else:
            api = 1

        if num_IP > 0:
            quals.append(ug_quals[-1])
            num_IP -= 1
          
        pilot = add_EXP_pilot(sorties, tos, api, quals)
        pilots.append(pilot)

    # Add INX pilots
    for _ in range(num_INX):
        tos = rng.integers(max_TOS_INX + 1)
        sorties = int(tos/max_TOS_INX * max_f16_sorties_INX)

        pilot = add_INX_pilot(sorties, tos, ug=ug_names[0])
        mqt_prog = rng.integers(ug_rides[0]) # Intentionally capping starting INX at 1 MQT ride from end
        for s in range(mqt_prog):
            pilot.fly_ug_sortie()

        pilots.append(pilot)

    return pilots

def print_sq(pilot_list):
    return ['Pilot {}: '.format(i) + p.summarize() for i, p in enumerate(sq)]










In [320]:
sq = populate_sq(10)
sq

In [321]:
print_sq(sq)

["Pilot 0: EXP-6 -- S: 296 -- T: 25mo. -- Q: ['WM', 'FL', 'IP']",
 "Pilot 1: EXP-6 -- S: 421 -- T: 29mo. -- Q: ['WM', 'FL']",
 "Pilot 2: EXP-1 -- S: 515 -- T: 32mo. -- Q: ['WM', 'FL']",
 "Pilot 3: EXP-1 -- S: 265 -- T: 24mo. -- Q: ['WM', 'FL']",
 "Pilot 4: EXP-1 -- S: 515 -- T: 32mo. -- Q: ['WM', 'FL']",
 'Pilot 5: (I)-1 -- S: 180 -- T: 15mo. -- Q: [] -- U: MQT #9',
 'Pilot 6: (I)-1 -- S: 109 -- T: 9mo. -- Q: [] -- U: MQT #1',
 'Pilot 7: (I)-1 -- S: 184 -- T: 16mo. -- Q: [] -- U: MQT #3',
 'Pilot 8: (I)-1 -- S: 134 -- T: 11mo. -- Q: [] -- U: MQT #5',
 'Pilot 9: (I)-1 -- S: 162 -- T: 14mo. -- Q: [] -- U: MQT #2']

In [322]:
sq[5].fly_ug_sortie()

In [323]:
print_sq(sq)

["Pilot 0: EXP-6 -- S: 296 -- T: 25mo. -- Q: ['WM', 'FL', 'IP']",
 "Pilot 1: EXP-6 -- S: 421 -- T: 29mo. -- Q: ['WM', 'FL']",
 "Pilot 2: EXP-1 -- S: 515 -- T: 32mo. -- Q: ['WM', 'FL']",
 "Pilot 3: EXP-1 -- S: 265 -- T: 24mo. -- Q: ['WM', 'FL']",
 "Pilot 4: EXP-1 -- S: 515 -- T: 32mo. -- Q: ['WM', 'FL']",
 "Pilot 5: (I)-1 -- S: 181 -- T: 15mo. -- Q: ['WM']",
 "Pilot 6: (I)-1 -- S: 109 -- T: 9mo. -- Q: ['WM'] -- U: MQT #1",
 "Pilot 7: (I)-1 -- S: 184 -- T: 16mo. -- Q: ['WM'] -- U: MQT #3",
 "Pilot 8: (I)-1 -- S: 134 -- T: 11mo. -- Q: ['WM'] -- U: MQT #5",
 "Pilot 9: (I)-1 -- S: 162 -- T: 14mo. -- Q: ['WM'] -- U: MQT #2"]

In [285]:
sq[4].enroll_upgrade('IPUG')
for s in range(6):
  sq[4].fly_ug_sortie()

print_sq(sq)

["Pilot 0: EXP-6 -- S: 390 -- T: 28mo. -- Q: ['WM', 'FL', 'IP']",
 "Pilot 1: EXP-6 -- S: 327 -- T: 26mo. -- Q: ['WM', 'FL']",
 "Pilot 2: EXP-1 -- S: 358 -- T: 27mo. -- Q: ['WM', 'FL']",
 "Pilot 3: EXP-1 -- S: 358 -- T: 27mo. -- Q: ['WM', 'FL']",
 "Pilot 4: EXP-1 -- S: 458 -- T: 30mo. -- Q: ['WM', 'FL'] -- U: IPUG #6",
 "Pilot 5: (I)-1 -- S: 101 -- T: 8mo. -- Q: ['WM'] -- U: MQT #3",
 "Pilot 6: (I)-1 -- S: 244 -- T: 22mo. -- Q: ['WM'] -- U: MQT #0",
 "Pilot 7: (I)-1 -- S: 94 -- T: 7mo. -- Q: ['WM'] -- U: MQT #7",
 "Pilot 8: (I)-1 -- S: 178 -- T: 15mo. -- Q: ['WM'] -- U: MQT #7",
 "Pilot 9: (I)-1 -- S: 275 -- T: 24mo. -- Q: ['WM']"]

In [296]:
sq[7].fly_ug_sortie()

In [297]:
print_sq(sq)

["Pilot 0: EXP-6 -- S: 390 -- T: 28mo. -- Q: ['WM', 'FL', 'IP']",
 "Pilot 1: EXP-6 -- S: 327 -- T: 26mo. -- Q: ['WM', 'FL']",
 "Pilot 2: EXP-1 -- S: 358 -- T: 27mo. -- Q: ['WM', 'FL']",
 "Pilot 3: EXP-1 -- S: 358 -- T: 27mo. -- Q: ['WM', 'FL']",
 "Pilot 4: EXP-1 -- S: 460 -- T: 30mo. -- Q: ['WM', 'FL', 'IP']",
 "Pilot 5: (I)-1 -- S: 101 -- T: 8mo. -- Q: ['WM', 'WM'] -- U: MQT #3",
 "Pilot 6: (I)-1 -- S: 244 -- T: 22mo. -- Q: ['WM', 'WM'] -- U: MQT #0",
 "Pilot 7: (I)-1 -- S: 97 -- T: 7mo. -- Q: ['WM', 'WM']",
 "Pilot 8: (I)-1 -- S: 178 -- T: 15mo. -- Q: ['WM', 'WM'] -- U: MQT #7",
 "Pilot 9: (I)-1 -- S: 275 -- T: 24mo. -- Q: ['WM', 'WM']"]